In [ ]:
!pip install rdkit-pypi
!pip install optuna

In [ ]:
!pip install --upgrade --force-reinstall \
  "numpy<2" \
  "scipy<1.12" \
  "scikit-learn<1.4" \
  rdkit-pypi

In [ ]:
import zipfile
import numpy as np
import pandas as pd
import os
import random
from rdkit import Chem
from rdkit.Chem import AllChem, DataStructs, Descriptors
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

CFG = {
    'NBITS': 2048,
    'SEED': 42,
    'N_SPLITS': 5,
    'N_TRIALS': 50
}

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(CFG['SEED'])

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

zip_path = "/content/drive/MyDrive/Colab Notebooks/Dacon/jumpAI2025/open.zip"
extract_path = "/content/"
os.makedirs(extract_path, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

chembl = pd.read_csv("ChEMBL_ASK1(IC50).csv", sep=';')
pubchem = pd.read_csv("Pubchem_ASK1.csv")

In [ ]:
# SMILES 데이터를 분자 지문으로 변환
def smiles_to_fingerprint(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is not None:
        fp = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=CFG['NBITS'])
        return np.array(fp)
    else:
        return np.zeros((CFG['NBITS'],))

In [ ]:
def IC50_to_pIC50(ic50_nM):
    ic50_nM = np.clip(ic50_nM, 1e-10, None)
    return 9 - np.log10(ic50_nM)
def pIC50_to_IC50(pIC50):
    return 10 ** (9 - pIC50)

In [ ]:
chembl.columns = chembl.columns.str.strip().str.replace('"', '')
chembl = chembl[chembl['Standard Type'] == 'IC50']
chembl = chembl[['Smiles', 'Standard Value']].rename(columns={'Smiles': 'smiles', 'Standard Value': 'ic50_nM'}).dropna()
chembl['ic50_nM'] = pd.to_numeric(chembl['ic50_nM'], errors='coerce')
chembl['pIC50'] = IC50_to_pIC50(chembl['ic50_nM'])

pubchem = pubchem[['SMILES', 'Activity_Value']].rename(columns={'SMILES': 'smiles', 'Activity_Value': 'ic50_nM'}).dropna()
pubchem['ic50_nM'] = pd.to_numeric(pubchem['ic50_nM'], errors='coerce')
pubchem['pIC50'] = IC50_to_pIC50(pubchem['ic50_nM'])

In [ ]:
df = pd.concat([chembl, pubchem], ignore_index=True)
df = df.drop_duplicates(subset='smiles')
df = df[df['ic50_nM'] > 0].dropna()
df['Fingerprint'] = df['smiles'].apply(smiles_to_fingerprint)
df = df[df['Fingerprint'].notnull()]

In [ ]:
# calculate_rdkit_descriptors
def calculate_rdkit_descriptors(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None: return np.full((len(Descriptors._descList),), np.nan)
    descriptors = [desc_func(mol) for _, desc_func in Descriptors._descList]
    return np.array(descriptors)

In [ ]:
def get_score(y_true_ic50, y_pred_ic50, y_true_pic50, y_pred_pic50):
    rmse = mean_squared_error(y_true_ic50, y_pred_ic50, squared=False)
    nrmse = rmse / (np.max(y_true_ic50) - np.min(y_true_ic50))
    A = 1 - min(nrmse, 1)
    B = r2_score(y_true_pic50, y_pred_pic50)
    score = 0.4 * A + 0.6 * B
    return score


TRAIN

In [ ]:
import lightgbm as lgb
import optuna

def objective(trial, X, y):
    params = {
        'objective': 'regression', 'metric': 'rmse', 'verbose': -1, 'n_jobs': -1,
        'seed': CFG['SEED'], 'boosting_type': 'gbdt', 'n_estimators': 2000,
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 20, 100),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.6, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.6, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 50),
    }

    kf = KFold(n_splits=CFG['N_SPLITS'], shuffle=True, random_state=CFG['SEED'])
    oof_preds = np.zeros(len(X))

    for train_idx, val_idx in kf.split(X, y):
        X_train, X_val = X[train_idx], X[val_idx]
        y_train, y_val = y[train_idx], y[val_idx]
        model = lgb.LGBMRegressor(**params)
        model.fit(X_train, y_train, eval_set=[(X_val, y_val)],
                  eval_metric='rmse', callbacks=[lgb.early_stopping(100, verbose=False)])
        oof_preds[val_idx] = model.predict(X_val)

    y_ic50_true = pIC50_to_IC50(y)
    oof_ic50_preds = pIC50_to_IC50(oof_preds)
    score = get_score(y_ic50_true, oof_ic50_preds, y, oof_preds)
    return score

In [ ]:
print("1. Loading and preprocessing data...")
train_df = df.copy()
if train_df is not None:
    train_df['pIC50'] = IC50_to_pIC50(train_df['ic50_nM'])
    print("\n--- Feature Engineering ---")
    train_df['fingerprint'] = train_df['smiles'].apply(smiles_to_fingerprint)
    train_df['descriptors'] = train_df['smiles'].apply(calculate_rdkit_descriptors)
    train_df.dropna(subset=['fingerprint', 'descriptors'], inplace=True)

    desc_stack = np.stack(train_df['descriptors'].values)
    desc_mean = np.nanmean(desc_stack, axis=0)
    desc_stack = np.nan_to_num(desc_stack, nan=desc_mean)

    scaler = StandardScaler()
    desc_scaled = scaler.fit_transform(desc_stack)
    fp_stack = np.stack(train_df['fingerprint'].values)
    X = np.hstack([fp_stack, desc_scaled])
    y = train_df['pIC50'].values

In [ ]:
print("\n--- Starting Hyperparameter Optimization with Optuna ---")
study = optuna.create_study(direction='maximize', study_name='lgbm_tuning')
study.optimize(lambda trial: objective(trial, X, y), n_trials=CFG['N_TRIALS'])

print(f"\nOptimization Finished. Best Score: {study.best_value:.4f}")
print("Best Parameters:", study.best_params)

best_params = { 'objective': 'regression', 'metric': 'rmse', 'verbose': -1, 'n_jobs': -1,
                'seed': CFG['SEED'], 'boosting_type': 'gbdt', 'n_estimators': 2000 }
best_params.update(study.best_params)

In [ ]:
print("\n--- Training Final Model with Best Parameters ---")
test_df = pd.read_csv("./test.csv")
test_df['fingerprint'] = test_df['Smiles'].apply(smiles_to_fingerprint)
test_df['descriptors'] = test_df['Smiles'].apply(calculate_rdkit_descriptors)

valid_test_mask = test_df['fingerprint'].notna() & test_df['descriptors'].notna()
fp_test_stack = np.stack(test_df.loc[valid_test_mask, 'fingerprint'].values)
desc_test_stack = np.stack(test_df.loc[valid_test_mask, 'descriptors'].values)
desc_test_stack = np.nan_to_num(desc_test_stack, nan=desc_mean)
desc_test_scaled = scaler.transform(desc_test_stack)
X_test = np.hstack([fp_test_stack, desc_test_scaled])

kf = KFold(n_splits=CFG['N_SPLITS'], shuffle=True, random_state=CFG['SEED'])
test_preds = np.zeros(len(X_test))

for fold, (train_idx, val_idx) in enumerate(kf.split(X, y)):
    print(f"--- Training Fold {fold+1}/{CFG['N_SPLITS']} ---")
    X_train, y_train = X[train_idx], y[train_idx]
    model = lgb.LGBMRegressor(**best_params)
    model.fit(X_train, y_train)
    test_preds += model.predict(X_test) / CFG['N_SPLITS']

SUBMISSION

In [ ]:
print("\n3. Generating submission file...")
submission_df = pd.read_csv("./sample_submission.csv")
pred_df = pd.DataFrame({'ID': test_df.loc[valid_test_mask, 'ID'], 'ASK1_IC50_nM': pIC50_to_IC50(test_preds)})
submission_df = submission_df[['ID']].merge(pred_df, on='ID', how='left')
submission_df['ASK1_IC50_nM'].fillna(train_df['ic50_nM'].mean(), inplace=True)
submission_df.to_csv("lgbm_tuned_submission.csv", index=False)
print("Submission file")